In [29]:
from allennlp.commands.elmo import ElmoEmbedder

options_file = "Elmo/data/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "Elmo/data/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

print("loadding elmo")
elmo = ElmoEmbedder(options_file, weight_file, cuda_device=0)
print("finish loading")

print("ok??????????????????????")
elmo_embedding = elmo.embed_sentence(["I", "Love", "You"])
print("ok!!!!!!!!!!!!!!!!!!!!")

Initializing ELMo.


loadding elmo
finish loading
ok??????????????????????
ok!!!!!!!!!!!!!!!!!!!!


In [2]:
# import pdb; pdb.set_trace()
# from config import Config
# from model.data_utils import CoNLLDataset
# from model.ner_model import NERModel
# import ray
# import ray.tune as tune

# config = Config()

# # build model
# model = NERModel(config)
# model.build()
# # model.restore_session("results/crf/model.weights/") # optional, restore weights
# #model.reinitialize_weights("proj")


# # create datasets


# dev   = CoNLLDataset(config.filename_dev, elmo, config.processing_word,
#                  config.processing_tag, config.max_iter)
# train = CoNLLDataset(config.filename_train, elmo, config.processing_word,
#                  config.processing_tag, config.max_iter)

# # train model
# model.train(train, dev)

In [32]:
def minibatches(data, minibatch_size, elmo):
    """
    Args:
        data: generator of (sentence, tags) tuples
        minibatch_size: (int)

    Yields:
        list of tuples

    """
    # import alog
    # alog.info("!!! I am here")
    x_batch, y_batch = [], []
    z_batch = []
    for (x, y, z) in data:
        if len(x_batch) == minibatch_size:
            activations, mask = elmo.batch_to_embeddings(z_batch)
            activations = np.transpose(activations, (0, 2, 1, 3))
            activations = activations.reshape(activations.shape[0],
                                              activations.shape[1],
                                              -1)
            z_batch = activations.cpu().numpy()
            # print("shape", z_batch.shape)
            yield x_batch, y_batch, z_batch.tolist()

            x_batch, y_batch = [], []
            z_batch = []

        if type(x[0]) == tuple:
            x = zip(*x)

        x_batch += [x]
        y_batch += [y]
        z_batch += [z]


    # z_batch = np.array(z_batch)
    # alog.info("!!! I am out")

    if len(x_batch) != 0:
        activations, mask = elmo.batch_to_embeddings(z_batch)
        activations = np.transpose(activations, (0, 2, 1, 3))
        activations = activations.reshape(activations.shape[0],
                                          activations.shape[1],
                                          -1)
        z_batch = activations.cpu().numpy()
        # print("shape", z_batch.shape)
        yield x_batch, y_batch, z_batch.tolist()


In [40]:
import numpy as np
dev = CoNLLDataset(config.filename_dev,
                   config.processing_word,
                   config.processing_tag,
                   config.max_iter)
train = CoNLLDataset(config.filename_train,
                     config.processing_word,
                     config.processing_tag,
                     config.max_iter)
test = CoNLLDataset(config.filename_test,
                     config.processing_word,
                     config.processing_tag,
                     config.max_iter, test=True)
batch_size = 10
out_file = "./elmo_test.embedding"

elmo_embedding_all = []
for i, (words, labels, elmo_embedding) in enumerate(minibatches(test, batch_size, elmo)):
    elmo_embedding_all.append(elmo_embedding)
np.savez(out_file, all_embedding)

In [11]:
# import pdb; pdb.set_trace()
import numpy as np
import copy
from tqdm import tqdm

out_file = "./elmo_train.embedding"
all_embedding = []
filename = "/SSD1/yinghong/tmp/s_t_elmo/data/train.eval.ebieo"
with open(filename) as f:
    words, tags = [], []
    orig_words = []
    # elmo_embedding = []
    for line in tqdm(f):
        line = line.strip()
        if (len(line) == 0 or line.startswith("-DOCSTART-")):
            if len(words) != 0:
                # niter += 1
                # if self.max_iter is not None and niter > self.max_iter:
                #     break
                # todo remote it
                # use_elmo = True
                # if use_elmo:
                embedding = elmo.embed_sentence(orig_words)
                embedding = np.transpose(embedding, (1, 0, 2))
                embedding = embedding.reshape(embedding.shape[0], -1)
                embedding = np.array(embedding)
                print(embedding.shape)
                all_embedding.append(embedding)
                # yield words, tags, self.elmo.embed_sentence(orig_words)
                words, tags = [], []
                orig_words = []
        else:
            ls = line.split(' ')
            word, tag = ls[0], ls[-1]
           
            words += [word]
            tags += [tag]
            orig_words += [word]
    all_embedding = np.array(all_embedding)
    np.savez(out_file, all_embedding)
    

68it [00:00, 302.63it/s]

(23, 3072)
(2, 3072)
(10, 3072)
(10, 3072)
(2, 3072)
(15, 3072)
(6, 3072)


152it [00:00, 347.63it/s]

(32, 3072)
(3, 3072)
(4, 3072)
(8, 3072)
(2, 3072)
(3, 3072)
(6, 3072)
(3, 3072)
(2, 3072)
(4, 3072)
(6, 3072)
(7, 3072)
(2, 3072)
(2, 3072)


263it [00:00, 382.55it/s]

...

705it [00:01, 394.33it/s]

(13, 3072)
(2, 3072)
(24, 3072)
(13, 3072)
(19, 3072)
(6, 3072)
(2, 3072)
(3, 3072)
(3, 3072)
(3, 3072)


762it [00:01, 399.79it/s]

(33, 3072)
(14, 3072)
(2, 3072)
(2, 3072)


KeyboardInterrupt: 

In [ ]:
# import pdb; pdb.seet_trace()

from config import Config
from model.data_utils import CoNLLDataset
from model.ner_model import NERModel
import ray
import ray.tune as tune
import tensorflow as tf

%load_ext autoreload
%autoreload 2

config = Config()

tf.reset_default_graph()
# build model
model = NERModel(config)
model.build()
# model.restore_session("results/crf/model.weights/") # optional, restore weights
#model.reinitialize_weights("proj")


# create datasets

# elmo = 1
dev   = CoNLLDataset(config.filename_dev, config.processing_word,
                     config.processing_tag, config.max_iter)
train = CoNLLDataset(config.filename_train, config.processing_word,
                     config.processing_tag, config.max_iter)

# train model
model.train(train, dev, elmo)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/yinghong/.pyenv/versions/pyenv-allennlp/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
Initializing tf session
Epoch 1 out of 50


shape (10, 32, 3072)
hahaha, I got it!!!!!!!!!
   1/1143 [..............................] - ETA: 839s - train loss: 23.6247shape (10, 8, 3072)
hahaha, I got it!!!!!!!!!
   2/1143 [..............................] - ETA: 514s - train loss: 15.4503shape (10, 39, 3072)
hahaha, I got it!!!!!!!!!
   3/1143 [..............................] - ETA: 469s - train loss: 22.2343shape (10, 21, 3072)
hahaha, I got it!!!!!!!!!
   4/1143 [..............................] - ETA: 430s - train loss: 20.7916shape (10, 25, 3072)
hahaha, I got it!!!!!!!!!
   5/1143 [..............................] - ETA: 401s - train loss: 18.5422shape (10, 24, 3072)
hahaha, I got it!!!!!!!!!
   6/1143 [..............................] - ETA: 383s - train loss: 17.6550shape (10, 24, 3072)
hahaha, I got it!!!!!!!!!
   7/1143 [..............................] - ETA: 366s - train loss: 16.9723shape (10, 33, 3072)
hahaha, I got it!!!!!!!!!
   8/1143 [..............................] - ETA: 363s - train loss: 15.9675shape (10, 20, 30

 134/1143 [==>...........................] - ETA: 269s - train loss: 7.5202shape (10, 19, 3072)
hahaha, I got it!!!!!!!!!
 135/1143 [==>...........................] - ETA: 268s - train loss: 7.4962shape (10, 29, 3072)
hahaha, I got it!!!!!!!!!
 136/1143 [==>...........................] - ETA: 268s - train loss: 7.5044shape (10, 32, 3072)
hahaha, I got it!!!!!!!!!
 137/1143 [==>...........................] - ETA: 268s - train loss: 7.5012shape (10, 26, 3072)
hahaha, I got it!!!!!!!!!
 138/1143 [==>...........................] - ETA: 268s - train loss: 7.5058shape (10, 51, 3072)
hahaha, I got it!!!!!!!!!
 139/1143 [==>...........................] - ETA: 269s - train loss: 7.5488shape (10, 10, 3072)
hahaha, I got it!!!!!!!!!
 140/1143 [==>...........................] - ETA: 268s - train loss: 7.5347shape (10, 13, 3072)
hahaha, I got it!!!!!!!!!
 141/1143 [==>...........................] - ETA: 267s - train loss: 7.5217shape (10, 20, 3072)
hahaha, I got it!!!!!!!!!
 142/1143 [==>..........

 268/1143 [======>.......................] - ETA: 233s - train loss: 6.0342shape (10, 15, 3072)
hahaha, I got it!!!!!!!!!
 269/1143 [======>.......................] - ETA: 232s - train loss: 6.0203shape (10, 56, 3072)
hahaha, I got it!!!!!!!!!
 270/1143 [======>.......................] - ETA: 232s - train loss: 6.0318shape (10, 43, 3072)
hahaha, I got it!!!!!!!!!
 271/1143 [======>.......................] - ETA: 233s - train loss: 6.0377shape (10, 16, 3072)
hahaha, I got it!!!!!!!!!
 272/1143 [======>.......................] - ETA: 232s - train loss: 6.0262shape (10, 26, 3072)
hahaha, I got it!!!!!!!!!
 273/1143 [======>.......................] - ETA: 232s - train loss: 6.0209shape (10, 25, 3072)
hahaha, I got it!!!!!!!!!
 274/1143 [======>.......................] - ETA: 232s - train loss: 6.0100shape (10, 40, 3072)
hahaha, I got it!!!!!!!!!
 275/1143 [======>.......................] - ETA: 232s - train loss: 6.0101shape (10, 19, 3072)
hahaha, I got it!!!!!!!!!
 276/1143 [======>......

 402/1143 [=========>....................] - ETA: 201s - train loss: 5.4021shape (10, 15, 3072)
hahaha, I got it!!!!!!!!!
 403/1143 [=========>....................] - ETA: 200s - train loss: 5.3945shape (10, 25, 3072)
hahaha, I got it!!!!!!!!!
 404/1143 [=========>....................] - ETA: 200s - train loss: 5.3920shape (10, 27, 3072)
hahaha, I got it!!!!!!!!!
 405/1143 [=========>....................] - ETA: 200s - train loss: 5.3879shape (10, 22, 3072)
hahaha, I got it!!!!!!!!!
 406/1143 [=========>....................] - ETA: 199s - train loss: 5.3822shape (10, 116, 3072)
hahaha, I got it!!!!!!!!!
 407/1143 [=========>....................] - ETA: 200s - train loss: 5.3863shape (10, 20, 3072)
hahaha, I got it!!!!!!!!!
 408/1143 [=========>....................] - ETA: 200s - train loss: 5.3939shape (10, 23, 3072)
hahaha, I got it!!!!!!!!!
 409/1143 [=========>....................] - ETA: 200s - train loss: 5.3876shape (10, 34, 3072)
hahaha, I got it!!!!!!!!!
 410/1143 [=========>..

 536/1143 [=============>................] - ETA: 163s - train loss: 4.9669shape (10, 14, 3072)
hahaha, I got it!!!!!!!!!
 537/1143 [=============>................] - ETA: 162s - train loss: 4.9600shape (10, 24, 3072)
hahaha, I got it!!!!!!!!!
 538/1143 [=============>................] - ETA: 162s - train loss: 4.9557shape (10, 28, 3072)
hahaha, I got it!!!!!!!!!
 539/1143 [=============>................] - ETA: 162s - train loss: 4.9496shape (10, 35, 3072)
hahaha, I got it!!!!!!!!!
 540/1143 [=============>................] - ETA: 162s - train loss: 4.9496shape (10, 36, 3072)
hahaha, I got it!!!!!!!!!
 541/1143 [=============>................] - ETA: 162s - train loss: 4.9629shape (10, 35, 3072)
hahaha, I got it!!!!!!!!!
 542/1143 [=============>................] - ETA: 161s - train loss: 4.9640shape (10, 27, 3072)
hahaha, I got it!!!!!!!!!
 543/1143 [=============>................] - ETA: 161s - train loss: 4.9560shape (10, 24, 3072)
hahaha, I got it!!!!!!!!!
 544/1143 [=============